# Lab 2: Quantization of AI models

## Intro

In this lab you will learn to **optimize** and **quantize AI models** using the **LiteRT** library (previously called Tensorflow Lite \[For Microcontrollers]). <br />

To be able to run the necessary scripts throughout this lab, you will need access to a GPU. You can either **make use of your own GPU** (through a Linux or Windows WSL system, with a GPU-enabled tensorflow installed (version 2.18.0)) **or use Google Colab**. <br />To run notebooks in colab, you will need to download the lab folder on Ufora, **unzip it and put it on your Google Drive** (this folder will only be a few MBs in size). You can **drag and drop** the unzipped folder in your Google Drive.<br /><br />


Next, **double click on the provided .ipynb file** for each lab which will open Google Colab. <br />From there, fill in the necessary variables (such as the path to your Google Drive) and you will be able to **run and program the necessary code. Be sure te select a GPU under Runtime > Change runtime type.**

In [1]:
%pip install --user --upgrade tensorflow-model-optimization
%pip install tf_keras

# Click Runtime > Restart session
# This ensures the above installed libraries are correctly imported

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Run this code to connect your Google Drive to Colab

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# Change to your project directory
# path_to_lab = "drive/MyDrive/Colab Notebooks/Embedded-ML-main/" # working on google colab
path_to_lab = "" # working locally

## Functions
Below you can find **functions** which can be used to complete the lab. <br />
_Note: when running the below code for the first time on Google Colab, you will get a warning that you need to restart your runtime session. This is expected because the kernel needs to use the expected tensorflow version._ 

In [3]:
import tensorflow as tf
from tensorflow import keras as keras
import tensorflow_model_optimization as tfmot
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

def mnist_model(train=False):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),
    tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(6, 6), activation=tf.nn.relu, name="conv1"),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu, name="conv2"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation=tf.nn.relu, name="dense1"),
    # tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax, name="dense2")
    ])

    model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    if train:
        model.fit(x=train_images, y= train_labels, batch_size=64, epochs=50, validation_data=(test_images, test_labels))
    else:
        # model = tf.keras.models.load_model("Models/mnist.keras")
        model = tf.keras.models.load_model(path_to_lab + "Models/mnist")
    return model


2025-03-28 15:06:31.394985: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 15:06:31.398550: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-28 15:06:31.408574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743170791.422490   27551 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743170791.426430   27551 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-28 15:06:31.441827: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Quantize models using LiteRT


### Part 1: Steps previous lab
1) Similar to the previous lab, load the mnist dataset and pre-trained model. For this exercise we will use a pre-trained model working on the mnist dataset for digit recognition.
2) Evaluate the model. To obtain a baseline performance, evaluate the model without any LiteRT optimizations applied.
3) Convert the model to the LiteRT format and evaluate whether this has an impact on performance or not.

In [4]:
# from code lab1: helper function to verify performance of tflite model
def verify_peformance(model_path):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=path_to_lab + model_path)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test model on random input data.
    # input_shape = input_details[0]['shape']
    # test_image = test_images[0].astype(np.float32)
    # test_image = np.expand_dims(test_image, axis=0)

    # interpreter.set_tensor(input_details[0]['index'], test_image)
    # interpreter.invoke()

    # output_data = interpreter.get_tensor(output_details[0]['index'])
    # predicted_label = np.argmax(output_data)

    correct = 0
    for i in range(len(test_images)):

        # change type of array elements form UINT to float32
        test_image = test_images[i].astype(np.float32)
        # change shape of test img to be batch of lenght 1
        test_image = np.expand_dims(test_image, axis=0)

        # input test_image
        interpreter.set_tensor(input_details[0]['index'], test_image)

        # run model
        interpreter.invoke()

        # get result
        output_data = interpreter.get_tensor(output_details[0]['index'])

        if np.argmax(output_data) == test_labels[i]:
            correct += 1

    accuracy = correct / len(test_images)
    model_name = model_path.split("/")[-1]
    print(f"TFLite Model ({model_name}) Accuracy: {accuracy:.4f}")

In [7]:
# Load dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Load pre-trained model
model = mnist_model(train=False) # same as lab0
# model.save("Models/mnist")

## Verify performance by inserting your code below

# ---- see lab 0

# Perform lite model conversion

# -- see lab 1: Part 1, for performance difference (there was none)
# Perform lite model conversion
converter = tf.lite.TFLiteConverter.from_saved_model(path_to_lab + 'Models/mnist') # path to the SavedModel directory
tflite_base_model = converter.convert()

with open(path_to_lab + 'Models/mnist_base.tflite', 'wb') as f:
  f.write(tflite_base_model)

verify_peformance(path_to_lab + 'Models/mnist_base.tflite')

W0000 00:00:1743170815.885128   27551 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743170815.885152   27551 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 15:06:55.885872: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: Models/mnist
2025-03-28 15:06:55.886847: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 15:06:55.886869: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: Models/mnist
I0000 00:00:1743170815.891115   27551 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
2025-03-28 15:06:55.891958: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 15:06:55.912915: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: Models/mnist
2025-03-28 15:06:55.920152: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tag

TFLite Model (mnist_base.tflite) Accuracy: 0.9912


### Part 2: New steps in this lab

In [5]:
# helper functions to check diff in models
def print_model_details(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    model_name = model_path.split("/")[-1]
    print("\n")
    print(f"Model: {model_name}")
    print(f"Number of tensors: {len(interpreter.get_tensor_details())}")
    print(f"Number of ops: {len(interpreter.get_signature_list())}")

    for tensor in interpreter.get_tensor_details()[0:3]:
        print(f"Tensor Name: {tensor['name']}, Shape: {tensor['shape']}, Type: {tensor['dtype']}")
    print("\n")

def check_weight_types(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    tensor_details = interpreter.get_tensor_details()
    weight_types = {tensor['dtype'] for tensor in tensor_details}

    model_name = model_path.split("/")[-1]
    print(f"Model: {model_name}")
    print(f"Weight data types: {weight_types}")
    print("\n")


4) Covert the model to the LiteRT format and **quantize the model** by enabling **dynamic range** quantization. (See [here](https://ai.google.dev/edge/litert/models/post_training_quantization))

In [7]:
# code for step 4.1

# Perform Dynamic-range quantization
dynamic_range_converter = tf.lite.TFLiteConverter.from_saved_model(path_to_lab + 'Models/mnist') # path to the SavedModel directory
dynamic_range_converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_dynamic_range_quant_model = dynamic_range_converter.convert()

with open(path_to_lab + 'Models/dynamic_range_model.tflite', 'wb') as f:
  f.write(tflite_dynamic_range_quant_model)

W0000 00:00:1743165629.923512    6279 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743165629.923539    6279 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 13:40:29.923737: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: Models/mnist
2025-03-28 13:40:29.924987: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 13:40:29.925006: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: Models/mnist
2025-03-28 13:40:29.931488: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 13:40:29.951106: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: Models/mnist
2025-03-28 13:40:29.958324: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 34593 microseconds.


In [19]:
# Verify performance
verify_peformance('Models/dynamic_range_model.tflite')

TFLite Model (dynamic_range_model.tflite) Accuracy: 0.9912


In [33]:
# Check model parameters compared to previous conversion
! ls -lh Models/mnist_base.tflite Models/dynamic_range_model.tflite

print_model_details("Models/mnist_base.tflite")
print_model_details("Models/dynamic_range_model.tflite")

check_weight_types("Models/mnist_base.tflite")
check_weight_types("Models/dynamic_range_model.tflite")

-rw-rw-r-- 1 jasper jasper  68K Mar 28 13:40 Models/dynamic_range_model.tflite
-rw-rw-r-- 1 jasper jasper 248K Mar 28 13:51 Models/mnist_base.tflite


Model: mnist_base.tflite
Number of tensors: 25
Number of ops: 1
Tensor Name: serving_default_input_4:0, Shape: [ 1 28 28], Type: <class 'numpy.float32'>
Tensor Name: arith.constant, Shape: [32], Type: <class 'numpy.float32'>
Tensor Name: arith.constant1, Shape: [64], Type: <class 'numpy.float32'>




Model: dynamic_range_model.tflite
Number of tensors: 25
Number of ops: 1
Tensor Name: serving_default_input_4:0, Shape: [ 1 28 28], Type: <class 'numpy.float32'>
Tensor Name: arith.constant, Shape: [2], Type: <class 'numpy.int32'>
Tensor Name: arith.constant1, Shape: [], Type: <class 'numpy.int32'>


Model: mnist_base.tflite
Weight data types: {<class 'numpy.float32'>, <class 'numpy.int32'>}


Model: dynamic_range_model.tflite
Weight data types: {<class 'numpy.int8'>, <class 'numpy.float32'>, <class 'numpy.int32'>}




In [17]:
# normalize to [0,1] for float32 consistency
train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)

In [18]:
# code for step 4.2
# Perform Full int8 quantization

# we need to stimate the range, i.e., (min, max) of all floating-point tensors in the model
def representative_dataset():
  indices = np.random.choice(len(train_images), 200, replace=False) # take evenely/randomly distrubted from data
  for i in indices:
      yield [np.expand_dims(train_images[i], axis=0)]

# set up converter
int8_converter = tf.lite.TFLiteConverter.from_saved_model(path_to_lab + 'Models/mnist') # path to the SavedModel directory
int8_converter.optimizations = [tf.lite.Optimize.DEFAULT]
int8_converter.representative_dataset = representative_dataset
int8_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] # only int8

# acutally convert and save
tflite_int8_quant_model = int8_converter.convert() 
with open(path_to_lab + 'Models/int8_model.tflite', 'wb') as f:
  f.write(tflite_int8_quant_model)

W0000 00:00:1743172453.576107   27551 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743172453.576135   27551 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 15:34:13.576361: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: Models/mnist
2025-03-28 15:34:13.578140: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 15:34:13.578162: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: Models/mnist
2025-03-28 15:34:13.585509: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 15:34:13.609302: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: Models/mnist
2025-03-28 15:34:13.620294: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 43935 microseconds.
fully_quantize: 0, inference_type: 6, input_inference_typ

In [69]:
# Verify performance
verify_peformance(path_to_lab + 'Models/int8_model.tflite')

TFLite Model (int8_model.tflite) Accuracy: 0.9914


**Q1: Do you see any difference in accuracy? What is changed in terms of the model parameters compared to the previous conversion?**
    
There is no change in accuracy as it stays 0.9912.

Simply comparing the size of the files we see a +- 3.5x reduction&nbsp;

- 68K Mar 28 13:40 Models/dynamic_range_model.tflite
- 248K Mar 28 13:51 Models/mnist_base.tflite

Taking a closer look at the parameters we see there is no difference in amount of tensors between the models but some tensor have different datatypes in the quantized model compared to base, conversion of some tensors from float/int32 to int8 lead to this reduction in size:

- Model: mnist_base.tflite \
Weight data types: {<class 'numpy.float32'="">, <class 'numpy.int32'="">}

- Model: dynamic_range_model.tflite \
Weight data types: {<class 'numpy.int8'="">, <class 'numpy.float32'="">, <class 'numpy.int32'="">}</class></class></class></class></class>
    
**Q2: Compared to dynamic range quantization, what accuracy difference do you get with full int8 precision quantization?**
There is a slight drop (accuracy = 0.9877) if we use only 100 images to represent the data.

If we use 200 images we can narrow the gap (accuracy = 0.9905).

And there is an even better result if we don't naively select the first 200 images. Instead I used `np.linspace` to try and represent the whole dataset better. This resulted in a slight performance boost (accuracy = 0.9910)

With random selection we can get even better results (accuracy = 0.9914), this random approach probably only works good if we use a big enough subset of the data.

This makes sense since we have to try and represent the range the value's in our dataset can take so the values of S and Z get estimated with more realistic r_min and r_max.

5) Try to train the model from scratch using **quantization-aware training.**

Full training from scratch

In [31]:
# code for step 5
# Perform Quantization aware training
model = mnist_model(train=False)
model = tf.keras.models.clone_model(model)  # we need weights from scratch

q_aware_model = tfmot.quantization.keras.quantize_model(model)

q_aware_model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])
# keep best fit
checkpoint = tf.keras.callbacks.ModelCheckpoint(path_to_lab + "Models/best_fits/q_aware_model", 
                    monitor="val_loss", mode="min", 
                    save_best_only=True, verbose=1)

# train from scratch
q_aware_model.fit(x=train_images, y= train_labels, batch_size=64, epochs=50, validation_data=(test_images, test_labels), callbacks=[checkpoint])

Epoch 1/50
937/938 [============================>.] - ETA: 0s - loss: 1.9020 - accuracy: 0.3141
Epoch 1: val_loss improved from inf to 1.86321, saving model to Models/best_fits/q_aware_model
INFO:tensorflow:Assets written to: Models/best_fits/q_aware_model/assets


INFO:tensorflow:Assets written to: Models/best_fits/q_aware_model/assets


938/938 [==============================] - 19s 19ms/step - loss: 1.9020 - accuracy: 0.3141 - val_loss: 1.8632 - val_accuracy: 0.2813
Epoch 2/50
929/938 [============================>.] - ETA: 0s - loss: 1.2567 - accuracy: 0.5430

In [ ]:
q_aware_model.load_weights(path_to_lab + "Models/best_fits/q_aware_model")
q_aware_model.save(path_to_lab + "Models/qat_mnist")

In [27]:
test_loss, test_acc = q_aware_model.evaluate(test_images, test_labels, verbose=2)

313/313 - 1s - loss: 0.0339 - accuracy: 0.9899 - 838ms/epoch - 3ms/step


In [28]:
# set up converter
int8_qat_converter = tf.lite.TFLiteConverter.from_saved_model(path_to_lab + "Models/qat_mnist") # path to the SavedModel directory
int8_qat_converter.optimizations = [tf.lite.Optimize.DEFAULT]
int8_qat_converter.representative_dataset = representative_dataset
int8_qat_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] # only int8

# acutally convert and save
tflite_int8_qat_model = int8_qat_converter.convert() 
with open(path_to_lab + 'Models/int8_qat_model.tflite', 'wb') as f:
  f.write(tflite_int8_qat_model)

W0000 00:00:1743174169.160433   27551 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743174169.160454   27551 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 16:02:49.160644: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: Models/qat_mnist
2025-03-28 16:02:49.164511: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 16:02:49.164540: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: Models/qat_mnist
2025-03-28 16:02:49.180340: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 16:02:49.245189: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: Models/qat_mnist
2025-03-28 16:02:49.266143: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 105503 microseconds.
fully_quantize: 0, inference_type: 6, input_

In [29]:
# Verify performance
verify_peformance('Models/int8_qat_model.tflite')

TFLite Model (int8_qat_model.tflite) Accuracy: 0.9898


Fine tuning model with 10 epochs

In [23]:
# code for step 5
# Perform Quantization aware training
model = mnist_model(train=False)

q_aware_model_fine_tune = tfmot.quantization.keras.quantize_model(model)

q_aware_model_fine_tune.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                      metrics=['accuracy'])
# train from scratch
q_aware_model_fine_tune.fit(x=train_images, y= train_labels, batch_size=64, epochs=10, validation_data=(test_images, test_labels))
q_aware_model_fine_tune.save(path_to_lab + "Models/qat_fine_time_mnist")

Epoch 1/10
938/938 [==============================] - 17s 17ms/step - loss: 0.2389 - accuracy: 0.9377 - val_loss: 0.0894 - val_accuracy: 0.9861
Epoch 2/10
938/938 [==============================] - 18s 19ms/step - loss: 0.0833 - accuracy: 0.9857 - val_loss: 0.0561 - val_accuracy: 0.9901
Epoch 3/10
938/938 [==============================] - 16s 17ms/step - loss: 0.0517 - accuracy: 0.9894 - val_loss: 0.0438 - val_accuracy: 0.9898
Epoch 4/10
938/938 [==============================] - 17s 18ms/step - loss: 0.0427 - accuracy: 0.9896 - val_loss: 0.0408 - val_accuracy: 0.9909
Epoch 5/10
938/938 [==============================] - 17s 18ms/step - loss: 0.0442 - accuracy: 0.9890 - val_loss: 0.0442 - val_accuracy: 0.9899
Epoch 6/10
938/938 [==============================] - 19s 20ms/step - loss: 0.0450 - accuracy: 0.9884 - val_loss: 0.0375 - val_accuracy: 0.9903
Epoch 7/10
938/938 [==============================] - 17s 18ms/step - loss: 0.0420 - accuracy: 0.9890 - val_loss: 0.0437 - val_accuracy:

INFO:tensorflow:Assets written to: Models/qat_fine_time_mnist/assets


In [30]:
test_loss, test_acc = q_aware_model_fine_tune.evaluate(test_images, test_labels, verbose=2)

313/313 - 1s - loss: 0.0384 - accuracy: 0.9903 - 915ms/epoch - 3ms/step


In [24]:
# set up converter
int8_qat_ft_converter = tf.lite.TFLiteConverter.from_saved_model(path_to_lab + "Models/qat_fine_time_mnist") # path to the SavedModel directory
int8_qat_ft_converter.optimizations = [tf.lite.Optimize.DEFAULT]
int8_qat_ft_converter.representative_dataset = representative_dataset
int8_qat_ft_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8] # only int8

# acutally convert and save
tflite_int8_qat_ft_model = int8_qat_ft_converter.convert() 
with open(path_to_lab + 'Models/int8_qat_ft_model.tflite', 'wb') as f:
  f.write(tflite_int8_qat_ft_model)

W0000 00:00:1743172973.088585   27551 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743172973.088606   27551 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 15:42:53.088859: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: Models/qat_fine_time_mnist
2025-03-28 15:42:53.092933: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 15:42:53.092954: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: Models/qat_fine_time_mnist
2025-03-28 15:42:53.113269: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 15:42:53.200226: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: Models/qat_fine_time_mnist
2025-03-28 15:42:53.228580: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 139728 microseconds.
fully_quantize

In [25]:
verify_peformance('Models/int8_qat_ft_model.tflite')

TFLite Model (int8_qat_ft_model.tflite) Accuracy: 0.9903


**Q3: What is the impact on accuracy with quantization-aware training?**

If we train a mnist model from scratch (clone before applying callingquantize_model(model)) with 50 epochs we get accuracy = 0.9898
If we train a mnnist model with fine-tuning of 10 epchs we get accuracy = 0.9903
Both are lower then the results without quantization-aware training which is not what i expected.

**Q4: When saving the tflite model, do you see any difference in the model size (full int8 quantization vs no quantization)?**
> amsw4

6) **Prune** the first three layers, at 85% AND perform **full INT8 quantization.**

In [ ]:
# code for step 6
# Perform pruning + quantization
# Verify performance


**Q5: Describe the observed effect in terms of accuracy and zipped model size when performing both pruning (first three layers, 85%) & full int8 quantization. (Tip: check the zipped tflite file size)**]
> answ5